## Uso del período de vida (TTL) de DynamoDB para caducar elementos

Período de vida (TTL) para Amazon DynamoDB le permite definir una marca temporal por elemento para determinar cuándo ya no se necesita un elemento. Poco después de la fecha y hora de la marca de tiempo especificadas, DynamoDB elimina el elemento de su tabla y solo consume rendimiento cuando la eliminación se replica a otras regiones. TTL se proporciona sin costo adicional como medio para reducir los volúmenes de datos almacenados reteniendo solo los elementos que permanecen actualizados para las necesidades de su carga de trabajo.

TTL es útil si almacena elementos que pierden relevancia después de un tiempo específico. A continuación, se muestran casos de uso de TTL de ejemplo:

* Elimine datos del usuario o del sensor después de un año de inactividad en una aplicación.
* Archive los elementos caducados en un lago de datos de Amazon S3 a través de Amazon DynamoDB Streams y AWS Lambda.
* Retenga la información confidencial durante un periodo determinado de tiempo de acuerdo con las obligaciones contractuales o reglamentarias.

Al habilitar TTL en una tabla, DynamoDB requiere que identifique un nombre de atributo específico que el servicio buscará al determinar si un elemento cumple los requisitos para el vencimiento. Además, otros requisitos garantizan que los procesos de TTL en segundo plano utilicen el valor del atributo de TTL. Si un elemento cumple los requisitos para el vencimiento a través de TTL:

* El elemento debe contener el atributo especificado cuando se habilitó TTL en la tabla. Por ejemplo, si especifica que una tabla utilice el nombre del atributo expdate como atributo de TTL, pero un elemento no tiene un atributo con ese nombre, el proceso de TTL ignora el elemento.

* El valor del atributo de TTL debe ser un tipo de datos Number de nivel superior. Por ejemplo, si especifica que una tabla utilice el nombre del atributo expdate como atributo de TTL, pero el atributo de un elemento es un tipo de datos String, los procesos de TTL ignoran el elemento.

* El valor del atributo de TTL debe ser una marca temporal en formato de tiempo Unix en segundos. Si utiliza cualquier otro formato, los procesos de TTL ignoran el elemento. Por ejemplo, si establece el valor del atributo en 1645119622, es decir, jueves 17 de febrero de 2022 17:40:22 h (GMT), el elemento vencerá después de ese momento. Para obtener ayuda para dar formato a las marcas de tiempo de fecha de inicio, puede utilizar herramientas de terceros, como Convertidor de fecha de inicio para obtener un formulario web visual.

* El valor del atributo de TTL debe ser una fecha y hora con un vencimiento de no más de cinco años en el pasado. Por ejemplo, si establece el valor del atributo en 1171734022, es decir, 17 de febrero de 2007 17:40:22 h (GMT) y más de cinco años. Como resultado, los procesos de TTL no vencen ese elemento.

### Notas de uso

Al usar TTL, tenga en cuenta lo siguiente:

* Habilitar, desactivar o cambiar la configuración de TTL en una tabla puede tardar aproximadamente una hora en propagarse la configuración y permitir la ejecución de cualquier otra acción relacionada con TTL.

* No se puede volver a configurar TTL para buscar un atributo diferente. Debe desactivar TTL y, a continuación, volver a habilitar TTL con el nuevo atributo en el futuro.

* Puede utilizar las políticas de AWS Identity and Access Management (IAM) para evitar actualizaciones no autorizadas del atributo de TTL en un elemento o la configuración de TTL. Si permite el acceso solo a acciones especificadas en las políticas de IAM existentes, asegúrese de que las políticas se actualizan para permitir dynamodb:UpdateTimeToLive para roles que necesitan habilitar o desactivar TTL en tablas. Para obtener más información, consulte Uso de políticas basadas en identidad (Políticas de IAM) para Amazon DynamoDB.

* Considere si necesita realizar cualquier procesamiento posterior de elementos eliminados a través de Amazon DynamoDB Streams, como archivar elementos en un lago de datos de Amazon S3. Los registros de Streams de eliminaciones de TTL destacan en una transmisión, ya que se marcan como eliminaciones del sistema en vez de eliminaciones normales. Puede filtrar estas eliminaciones del sistema y procesarlas posteriormente mediante una combinación de filtros de eventos de AWS Lambda y una función de AWS Lambda. Para obtener más información acerca de las incorporaciones a los registros de transmisiones, consulte Amazon DynamoDB Streams y Período de vida.

* Si la recuperación de datos es un problema, le recomendamos que realice una copia de seguridad de las tablas.

    * Para backup de tablas totalmente administradas, utilice on-demand backups (backup en diferido) or continuous backups with point-in-time recovery (Backup continuo con recuperación a un momento dado) de DynamoDB.

    * Para una periodo de recuperación de 24 horas, puede utilizar Amazon DynamoDB Streams. Para obtener más información, consulte Amazon DynamoDB Streams y período de vida.

In [11]:
import boto3
import uuid
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime, timedelta
import time
import random

In [4]:
dt=DynamoTable()
try:
    dt.select_table('SessionData')
    print(dt)
except:
    dt.create_table(
        table_name='SessionData',
        partition_key='UserName',
        partition_key_type='S',
        sort_key='SessionId',
        sort_key_type='S'
    )

Table created successfully!


In [31]:
today = datetime.now().strftime('%Y-%m-%d')
def generate_data(total_items=300):
    count_total = 0  
    UserName, SessionId, CreationTime, ExpirationTime, SessionInfo = [], [], [], [], []
    for i in range(total_items):
        if count_total == total_items:
            break
        else:
            count_total += 1
            now_time = datetime.now()
            # TTL in 7 days
            ttl_time = now_time + timedelta(days=7)
            uuid_session = str(uuid.uuid4())
            UserName.append('user'+str(random.randint(1, 100)))
            SessionId.append(uuid_session)
            CreationTime.append(int(now_time.timestamp()))
            ExpirationTime.append(int(ttl_time.timestamp()))
            SessionInfo.append({'SessionId': uuid_session, 'UserName': 'user'+str(random.randint(1, 100)), 'Date': today})
        
    df_main = pd.DataFrame(
        {
            'UserName': UserName,
            'SessionId': SessionId,
            'CreationTime': CreationTime,
            'ExpirationTime': ExpirationTime,
            'SessionInfo': SessionInfo
        }
    )   
    return df_main

In [32]:
df = generate_data(total_items=300)
dt.batch_pandas(df)

### Habilitar TTL en la tabla

In [33]:
client_dynamodb = boto3.client('dynamodb', region_name='us-east-1')

In [36]:
client_dynamodb.update_time_to_live(
    TableName=dt.table_name,
    TimeToLiveSpecification={
        'Enabled': True,
        'AttributeName': 'ExpirationTime'
    }
)

{'TimeToLiveSpecification': {'Enabled': True,
  'AttributeName': 'ExpirationTime'},
 'ResponseMetadata': {'RequestId': 'O4032NV04M2J3AOI3FL4I4I6PRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 23 Jun 2023 23:16:50 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '77',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'O4032NV04M2J3AOI3FL4I4I6PRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1019368986'},
  'RetryAttempts': 0}}